# Topic 1: Introduction to LangGraph

Welcome to your LangGraph journey! In this notebook, you'll learn the fundamentals of LangGraph and build your first graph.

## What is LangGraph?

LangGraph is a library for building stateful, multi-actor applications with LLMs. It's built on top of LangChain and adds the ability to create cyclical graphs, which are essential for agent-like behaviors.

### Key Concepts:

1. **Nodes**: Functions that do the work
2. **Edges**: Connections between nodes
3. **State**: Data that flows through the graph
4. **Graph**: The overall structure that orchestrates everything

## Setup

First, let's install the required packages and set up our environment.

In [ ]:
# Install required packages
!pip install langgraph langchain-anthropic langchain-core -q

In [ ]:
import os
import getpass

# Set your Anthropic API key
if "ANTHROPIC_API_KEY" not in os.environ:
    os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key: ")

print("✓ API key set successfully!")

## Core Concepts: State

In LangGraph, state is represented as a TypedDict. This defines what data flows through your graph.

In [ ]:
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

# Define our state
class State(TypedDict):
    messages: Annotated[list, add_messages]
    
print("State definition created!")
print("Our state has a 'messages' field that will accumulate messages.")

## Core Concepts: Nodes

Nodes are functions that receive the current state and return updates to that state.

In [ ]:
from langchain_anthropic import ChatAnthropic

# Initialize the Claude model
model = ChatAnthropic(model="claude-3-5-sonnet-20241022")

# Create a simple node that calls the LLM
def chatbot(state: State):
    """A node that calls Claude with the current messages."""
    response = model.invoke(state["messages"])
    return {"messages": [response]}

print("✓ Chatbot node created!")

## Building Your First Graph

Now let's combine everything to create a simple graph.

In [ ]:
# Create a new graph
graph_builder = StateGraph(State)

# Add our chatbot node
graph_builder.add_node("chatbot", chatbot)

# Add edges
graph_builder.add_edge(START, "chatbot")  # Entry point
graph_builder.add_edge("chatbot", END)    # Exit point

# Compile the graph
graph = graph_builder.compile()

print("✓ Graph compiled successfully!")

## Visualizing the Graph

LangGraph can generate a visual representation of your graph:

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    print("Graph visualization not available. Install graphviz to see the graph.")
    print("Graph structure: START -> chatbot -> END")

## Running the Graph

Let's test our graph with a simple message:

In [ ]:
from langchain_core.messages import HumanMessage

# Run the graph with a message
result = graph.invoke({
    "messages": [HumanMessage(content="Hello! What is LangGraph?")]
})

# Print the response
print("\nClaude's response:")
print(result["messages"][-1].content)

## Streaming Responses

LangGraph supports streaming, which is great for real-time applications:

In [ ]:
# Stream the graph execution
print("\nStreaming response:")
for event in graph.stream({
    "messages": [HumanMessage(content="Explain what makes LangGraph useful in 2 sentences.")]
}):
    for value in event.values():
        print(value["messages"][-1].content)

## Exercise: Build Your Own Graph

Now it's your turn! Create a graph that:
1. Takes a user message
2. Calls Claude
3. Returns the response

Try asking it to help you plan a simple application.

In [ ]:
# Your code here!
# Hint: You can reuse the code above and modify the message

your_message = "Help me plan a simple todo list application"

# TODO: Run the graph with your message


## Key Takeaways

In this notebook, you learned:

1. ✅ What LangGraph is and its core concepts (nodes, edges, state)
2. ✅ How to define state using TypedDict
3. ✅ How to create nodes that process state
4. ✅ How to build and compile a graph
5. ✅ How to run graphs and stream responses

## Next Steps

In the next notebook, you'll learn how to:
- Create more complex graphs with multiple nodes
- Manage state across multiple steps
- Build linear workflows

Continue to **Topic 2: Basic Graphs** when you're ready!